# Limpieza y creación CSV para Flourecer

**Carga de librerías**

In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

##### Carga Dataset Temperaturas

In [167]:
#Cargamos el df
temp = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
#Ajustamos la columna dt y creamos una nueva con los años 
temp = temp.rename(columns={'dt':'Date'})
temp['Date'] = pd.to_datetime(temp['Date'])
temp['Year'] = temp['Date'].dt.year 
temp = temp.drop(['Date'], axis=1)
#Eliminamos todos los nulos y reindexamos los índices
temp = temp.dropna()
temp.index = range(temp.shape[0])
#Se realizan renombremientos de valores para la implementación en Flourish
temp['Country'] = temp['Country'].replace(['United States'], 'United States of America')

#### Carga resto de dataframes (World Bank Data Base)

In [131]:
#Cargamos los datasets eliminando las 4 primeras filas que contienen espacios en blanco
CO2 = pd.read_csv('CO2.csv', skiprows=4)
NoRenovables = pd.read_csv('ProdEPGC.csv', skiprows=4)
renovables = pd.read_csv('PreduccionERenovable.csv', skiprows=4)
nuclear = pd.read_csv('Energia_Nuclear.csv', skiprows=4)
Population = pd.read_csv('Poblacion.csv', skiprows=4)
consumo = pd.read_csv('Consumo de energía eléctrica (kWh per cápita).csv', skiprows=4)
CO2.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Emisiones de CO2 (kt),EN.ATM.CO2E.KT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AFE,Emisiones de CO2 (kt),EN.ATM.CO2E.KT,NaN,NaN,NaN,NaN,NaN,NaN,...,565430.012964,587699.992426,574029.994987,578510.015696,587509.992346,590040.005282,603030.016154,NaN,NaN,NaN


Vemos como en CO2 desde 1960 a 1989 y de 2020 a 2021 no hay registros por lo que procedemos a eliminar esas columnas vacías.
 Creamos una función *limpieza* que elimina las columnas que no necesiamos del dataset y reestrablece índices. Esta aplica la función normalize para eliminar los accentos de los nombres de los países ya que estos están escritos en español. Añadir que se realizará manualmente la selección de países como veremos más adelante por este cambio de lengua entre df.
 
 Lo mismo ocurre para los dataframes Renovables, NoRenovables, nuclear... Los cuales son obtenidos de la misma base de datos data.worldbank.org.

In [132]:
def limpieza(df):
    #Realizamos un proceso de limpieza para ello eliminamos lad filas que no nos interesan 
    df = df.drop(['Unnamed: 66', 'Indicator Code', 'Country Code', 'Indicator Name'], axis=1)
    #Eliminamos las filas que son Na
    df = df.dropna(subset=['Country Name'])
    # Columnas de años que carecen de datos
    rdelete = ['1960','1961','1962','1963','1964','1965','1966','1967','1968','1969', '1970','1971','1972','1973','1974','1975',
           '1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987','1988', '1989', '2016', '2017',
               '2018','2019', '2020','2021']
    df = df.drop(rdelete, axis=1)
    df = df.fillna(0)
    #Eliminamos todos los valores nulos restantes
    df = df.dropna()
    #Renombramos los índices
    df.index = range(df.shape[0])
    
    return df

In [133]:
CO2 = limpieza(CO2)
NoRenovables = limpieza(NoRenovables)
renovables = limpieza(renovables)
nuclear = limpieza(nuclear)
Population = limpieza(Population)

 Para el df de consumo deja de haber valores a partir de 2015, por lo que dicho valor se rellenará con el valor del año 2014, para conseguir tener df de igual longitud temporal.

In [134]:
#APlicamos procesos de limpieza similares a la funcion lipieza
consumo = consumo.drop(['Unnamed: 66', 'Indicator Code', 'Country Code', 'Indicator Name'], axis=1)
consumo = consumo.dropna(subset=['Country Name'])
rdelete = ['1960','1961','1962','1963','1964','1965','1966','1967','1968','1969', '1970','1971','1972','1973','1974','1975',
           '1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987','1988', '1989', '2016', '2017',
               '2018','2019', '2020','2021']
consumo = consumo.drop(rdelete, axis=1)
consumo.index = range(consumo.shape[0])
for i in range(consumo.shape[0]):
    consumo['2015'][i] = consumo['2014'][i]
consumo = consumo.fillna(0)

Convertimos a csv para poder utilizarlos y crear representaciones en Flourish

In [135]:
CO2.to_csv('Flourecer/CO2.csv', index = False)
NoRenovables.to_csv('Flourecer/NoRenovables.csv', index = False)
renovables.to_csv('Flourecer/Renovables.csv', index = False)
nuclear.to_csv('Flourecer/Enuclear.csv', index = False)
Population.to_csv('Flourecer/Poblacion.csv', index = False)
consumo.to_csv('Flourecer/Consumo(Kwh).csv', index = False)

## Preguntas y representaciones

  Las preguntas a responder son:
  
     + ¿Es el incremento de la T. media de cada país directamente proporcional a su emisión de CO2?

     + ¿Cómo ha evolucionado la apuesta por las energías renovables para reducir la emisión de CO2?

     + Los países cuya principal fuente de energía es renovable ¿Han conseguido reducir sus emisiones de CO2?

### Intro

**Evolución CO2 (kt) y el consumo de energía eléctrica (Kwh)**

In [170]:
#Evolución CO2 respecto de su concentración en 1990

#Lista años 1990-2015
years = list(CO2.columns.values)[1:]

co2evolution = []
for y in years:
    co2evolution.append(sum(CO2[y])/sum(CO2['1990'])-1)
                              
CO2ev = pd.DataFrame() 
CO2ev['% CO2'] = co2evolution

CO2ev.index = years
CO2ev.to_csv('Flourecer/CO2ev.csv')

In [171]:
Consumoevolution = []
for y in years:
    Consumoevolution.append(sum(consumo[y])/sum(consumo['1990'])-1)
                              
Consumoev = pd.DataFrame() 
Consumoev['% Consumo'] = Consumoevolution

Consumoev.index = years
Consumoev.to_csv('Flourecer/Consumoev.csv')

**Incremento T. medias anuales de 1990 a 2015**

In [172]:
#Obtenemos los nombres de los países y el promedio de su temperatura
countries = np.unique(temp['Country'])

mean = []
for country in countries:
    mean.append(round(temp[temp['Country'] == country]['AverageTemperature'].mean(), 2))

    #Cargamos el df temp para usarlo en Flourish
TempGlobalbyCountry  = pd.DataFrame()
TempGlobalbyCountry['Pais'] = countries
TempGlobalbyCountry['T. Media'] = mean

TempGlobalbyCountry.to_csv('Flourecer/TempGlobalbyCountry.csv', index = False)

In [173]:
#Mapa incremento de T. Promedio
data = [ dict(
        type = 'choropleth',
        locations = countries,
        z = mean,
        locationmode = 'country names',
        text = countries,
        marker = dict(
            line = dict(color = 'rgb(0,0,0)', width = 1)),
            colorbar = dict(autotick = True, tickprefix = '',
            title = '# Temperatura\npromedio,\n°C'),
        #The following line is also needed to create Stream
        #stream = stream_id
            )
       ]

layout = dict(
    title = 'Promedio temperaturas paises coincidentes desde 1990 a 2015',
    geo = dict(
        showframe = False,
        showocean = True,
        oceancolor = 'rgb(0,255,255)',
        type = 'equirectangular'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='world_temp_map')

Para poder responder a una de las preguntas planteadas:
 
   + ¿Es el incremento de la T. media de los países directamente proporcional con su emisión de CO2?
   
Vamos a seleccionar los 5 países con mayor incremento de temperaturas tanto positivas como negativas.

In [82]:
mean_sorted, countries_sorted = (list(x) for x in zip(*sorted(zip(mean, countries), 
                                                             reverse = True)))

In [83]:
#Eliminamos Aruba por inconcluencia
max = countries_sorted[:6]
max.pop(4)
print('Los 5 países que sufren un mayor incremento de temperatura son:', max)

#Eliminamos una región noruega que aparece como país
less = countries_sorted[-6:]
less.pop(3)
print('Los 5 países que sufren un menor incremento o descenso de la temperatura son:', less)

Los 5 países que sufren un mayor incremento de temperatura son: ['Djibouti', 'Mali', 'Burkina Faso', 'Senegal', 'United Arab Emirates']
Los 5 países que sufren un menor incremento o descenso de la temperatura son: ['Mongolia', 'Canada', 'Russia', 'Denmark', 'Greenland']


 Vamos a crear un csv que sea emisiones de CO2 por número de habitantes para poder normalizar la variable. 
 A la hora de realizar el cálculo de kt de CO2/ Nº hab. kt = 10^6 g y el numde habitantes de cada país está en miles 10^3, para calcularlo solo debemos multiplicar por 10^3 el contenido de CO2

In [84]:
#Lista países con incrementos de temperaturas extremos 
extrem_countries = ['Djibouti', 'Mali', 'Burkina Faso', 'Senegal','Emiratos Arabes Unidos', 'Mongolia', 'Canada',
                    'Federacion de Rusia', 'Dinamarca', 'Groenlandia']

#Lista años 1990-2015
years = list(CO2.columns.values)[1:]

#Creamos un nuevo df que contenga los paises con incrementos más extremos
df_extreme = CO2[CO2['Country Name'].isin(extrem_countries)]
df_extreme.index = range(df_extreme.shape[0])

#Calculamos el valor de kt de Co2/Nº Hab
for año in years:
    for p in extrem_countries:
        df_extreme[año][df_extreme['Country Name'] == p] = df_extreme[año][df_extreme['Country Name'] == p].values.astype(np.float)*1000/Population[año][Population['Country Name'] == p].values.astype(np.float)

In [85]:
df_extreme

,Country Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Burkina Faso,0.054756,0.054458,0.054175,0.053861,0.053512,0.055055,0.059315,0.063318,0.067851,...,0.086615,0.095547,0.111857,0.110543,0.142087,0.148169,0.171792,0.177474,0.175015,0.197670
1,Djibouti,0.467797,0.461401,0.481389,0.490813,0.465133,0.508244,0.465370,0.494170,0.561216,...,0.507706,0.543230,0.532345,0.521583,0.565710,0.512377,0.523946,0.576278,0.404413,0.457139
2,Dinamarca,10.132390,12.184006,11.028025,11.440404,12.139419,11.464881,13.934822,12.043540,11.257077,...,10.691023,9.779476,9.125129,8.678105,8.673891,7.737087,6.835287,7.116738,6.347153,5.929463
3,Groenlandia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Mongolia,6.000649,6.708856,5.750071,5.219549,4.484013,4.411385,3.686838,3.577170,3.554875,...,4.811637,4.996847,4.924281,5.114911,5.295058,5.732637,6.138201,6.460109,6.242200,5.835232
5,Senegal,0.309183,0.300479,0.322277,0.306226,0.310087,0.319715,0.331320,0.354636,0.388705,...,0.494523,0.536153,0.542410,0.546944,0.561048,0.587921,0.563036,0.589163,0.613444,0.647108


In [86]:
#Trasponemos las filas y columnas para el gráfico lineal de Flourecer y creamos el csv.
df_extreme = df_extreme.transpose().reset_index()
df_extreme.to_csv('Flourecer/CO2_pob.csv', index = False)

 Creamos el csv para el gráfico de barras que muestra el porcentaje de fuente de producción de energía en cada año

In [174]:
percNR = []
percR = []
for y in years:
    percNR.append(round(sum(NoRenovables[y])/(sum(NoRenovables[y])+sum(renovables[y]))*100, 2))
    percR.append(round(sum(renovables[y])/(sum(NoRenovables[y])+sum(renovables[y]))*100, 2))

PercRyNR = pd.DataFrame() 
PercRyNR['No Renovable'] = percNR
PercRyNR['Renovable'] = percR
PercRyNR.index = years
PercRyNR

,No Renovable,Renovable
1990,57.98,42.02
1991,57.67,42.33
1992,58.10,41.90
1993,57.17,42.83
1994,57.22,42.78
1995,58.08,41.92
1996,58.22,41.78
1997,59.38,40.62
1998,59.76,40.24
1999,59.64,40.36


In [175]:
PercRyNR.to_csv('Flourecer/PercRyNR.csv')

**¿Los países que han tenida como principal fuente de energía las renovables han conseguido reducir sus emisiones de CO2?**

In [159]:
#Seleccionamos los 10 primeros países que han apostado más como principal fuente de energía a las renovables
most_ecologic = countries_sortedR[:8]

#Observamos la diferencia del CO2 emitido desde 1990 a 2015
for p in most_ecologic:
    a = np.float(CO2[CO2['Country Name'] == p]['1990'].values)
    b = np.float(CO2[CO2['Country Name'] == p]['2015'].values)
    r = (b-a)/(b+a)
    print(p, round(r, 3))

Paraguay 0.509
Bhután 0.812
Islandia 0.027
Congo, República Democrática del 0.008
Zambia 0.288
Noruega 0.148
Tayikistán -0.384
Nepal 0.769


In [164]:
#Creamos el df para trabajar en Flourish
df_green= CO2[CO2['Country Name'].isin(most_ecologic)]
df_green.index = range(df_green.shape[0])
df_green = df_green.transpose().reset_index()
df_green.to_csv('Flourecer/green.csv', index = False)

In [165]:
df_green

,index,0,1,2,3,4,5,6,7
0,Country Name,Bhután,"Congo, República Democrática del",Islandia,Noruega,Nepal,Paraguay,Tayikistán,Zambia
1,1990,80.0,3180.0,1950.0,29310.0,940.0,2070.0,11020.0,2740.0
2,1991,130.0,1740.0,1910.0,27330.0,1180.0,2020.0,10170.0,2880.0
3,1992,120.0,1480.0,1940.0,30200.0,1230.0,2470.0,7690.0,2850.0
4,1993,130.0,1900.0,1980.0,32430.0,1410.0,2820.0,5170.0,2510.0
5,1994,140.0,2210.0,2090.0,34410.0,1740.0,3340.0,3040.0,2140.0
6,1995,160.0,2110.0,2000.0,34390.0,1900.0,3780.0,2510.0,2130.0
7,1996,200.0,2270.0,2270.0,34600.0,1960.0,3720.0,2240.0,1760.0
8,1997,190.0,2270.0,2150.0,36540.0,2190.0,4110.0,2510.0,2290.0
9,1998,200.0,2280.0,2150.0,37890.0,2320.0,4270.0,2770.0,2170.0
